In [1]:
import pandas as pd
import numpy as np
import pymysql
import time
import datetime
import joblib
import re
import difflib
import copy
import requests
import json

In [ ]:
# def sql_iddata():
#     conn = pymysql.connect(
#         host="

#     user='wuboyuan',
#     password='',
#         db="api_loansuper",
#         charset="utf8")

#     sql2 = f"""
# select a.loan_id,a.is_new,a.mobile,a.id_card_no, a.request_date, a.app_user_id,a.app_user_name,a.repay_order_status
# from t_loan_order a
# left join t_loan_order_operate b
# on a.loan_id = b.loan_id
# where
# a.request_date>='2021-06-01'
# and
# a.request_date<='2021-07-01'
# and
# allplatfrom_user_type=3
# and
# loan_order_status=5

#     """
#     df = pd.read_sql(sql2, conn)
#     #df = df.drop_duplicates("app_user_id")
#     conn.close()
#     return df

# iddata=sql_iddata()

In [164]:
def multipe_loan(db=None,id_card_no=None):
    conn = pymysql.connect(
        host="",
        user="wuboyuan",
        password="",
        db=f"{db}",
        charset="utf8")

    sql=f"""
 SELECT
            a.loan_id,
            a.request_date,
            b.repay_date,
            a.repay_order_status,
            a.allplatfrom_user_type,
            a.item_code,
            a.id_card_no,
            a.mobile,
            a.extend_times,
            a.loan_order_status,
            a.remaining_days,
            b.sell_accounts_date,
						b.push_cash_date,
						a.loan_cycle,
						a.loan_amount
        FROM
            t_loan_order a
            LEFT JOIN t_loan_order_operate b ON a.loan_id = b.loan_id 
        WHERE
            1 = 1 
						and
						a.id_card_no="{id_card_no}"
					UNION
        SELECT
            loan_id,
            request_date,
            repay_date,
            1 AS 'repay_order_status',
            CAST( JSON_EXTRACT( loan_order_json, '$.allplatfromUserType' ) AS SIGNED ) AS 'allplatfrom_user_type',
            item_code,
            id_card_no,
            mobile,
            CAST( JSON_EXTRACT( loan_order_json, '$.extendTimes' ) AS SIGNED ) AS 'extend_times',
            5 AS 'loan_order_status',
            postpone_day AS 'remaining_days', 
            create_date AS 'sell_account_date' ,
						push_cash_date,
						  CAST( JSON_EXTRACT( loan_order_json, '$.loanCycle' ) AS SIGNED ) AS 'loan_cycle',
							CAST( JSON_EXTRACT( loan_order_json, '$.loanAmount' ) AS SIGNED ) AS 'loan_amount'
        FROM
            t_loan_order_postpone_log 
					WHERE
					id_card_no="{id_card_no}"

   """              
    df=pd.read_sql(sql,conn)
    conn.close()
    return df

In [165]:
def multipe_data(id_card_no=None):
    ind=0
    for db in ['api_loansuper','api_loansuper_x2','api_loansuper_x4','api_loansuper_x5']:
        if ind==0:
            multipe_loan_set1=multipe_loan(db=db,id_card_no=id_card_no)
            ind=1
        else:
            multipe_loan_set1=multipe_loan_set1.append(multipe_loan(db=db,id_card_no=id_card_no))
    return multipe_loan_set1

In [166]:
multipeloan=multipe_data(id_card_no=3214014707970001)

In [219]:
def count_multipe(multipeloan=None,requests_date_y=None,loan_id=None):
    multipeloan_result={}
    multipeloan1=multipeloan[~multipeloan['push_cash_date'].isnull()]
    multipeloan1['push_cash_date']=pd.to_datetime(multipeloan1['push_cash_date'])
    for day_y in [0.1,1,3,5,7,10,365]:
        data_set=multipeloan[(multipeloan['request_date']>requests_date_y+pd.Timedelta(days=-day_y)) &(multipeloan['request_date']<requests_date_y)]
        multipeloan
        data_set2=multipeloan1[(multipeloan1['push_cash_date']>requests_date_y+pd.Timedelta(days=-day_y)) &(multipeloan1['push_cash_date']<requests_date_y)]

        multipeloan_result['loan_request_'+str(day_y)]=len(data_set)
        multipeloan_result['loan_push_'+str(day_y)]=len(data_set2[data_set2['loan_order_status']==5])
        multipeloan_result['remain_loan_amount_'+str(day_y)]=data_set2[data_set2['loan_order_status']==5]['loan_amount'].sum()
        multipeloan_loanid=pd.DataFrame(multipeloan_result,index=[loan_id])
    return multipeloan_loanid

In [231]:
def multipe_result(id_card_no=None,requests_date_y=None,loan_id=None):
    multipeloan=multipe_data(id_card_no=id_card_no)
    count_multipe_loan=count_multipe(multipeloan=multipeloan,requests_date_y=requests_date_y,loan_id=loan_id)
    return count_multipe_loan

In [235]:
multipe_result(id_card_no='1105014409890001',requests_date_y=iddata['request_date'].iloc[i],loan_id=977796)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,loan_request_0.1,loan_push_0.1,remain_loan_amount_0.1,loan_request_1,loan_push_1,remain_loan_amount_1,loan_request_3,loan_push_3,remain_loan_amount_3,loan_request_5,...,remain_loan_amount_5,loan_request_7,loan_push_7,remain_loan_amount_7,loan_request_10,loan_push_10,remain_loan_amount_10,loan_request_365,loan_push_365,remain_loan_amount_365
977796,0,0,0.0,0,0,0.0,7,4,10290000.0,11,...,17300000.0,15,9,21600000.0,20,14,32050000.0,29,23,48660000.0


In [240]:
1

1

In [241]:
def sql_iddata():
    conn = pymysql.connect(
        host="
    user='wuboyuan',
    password='',
        db="api_loansuper",
        charset="utf8")

    sql2 = f"""
SELECT
*
from
t_app_user_connections
WHERE
create_date>='2021-01-01'
and
create_date<='2021-02-01'

    """
    df = pd.read_sql(sql2, conn)
    #df = df.drop_duplicates("app_user_id")
    conn.close()
    return df

iddata=sql_iddata()

KeyboardInterrupt: 